# PySpark vs pandas syntax

In [1]:
from pathlib import Path

import delta
import pyspark
import pyspark.sql.functions as F
from delta import configure_spark_with_delta_pip

## PySpark syntax

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.executor.memory", "10G")
    .config("spark.driver.memory", "25G")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.sql.shuffle.partitions", "2")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-340-delta-240/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c7781fcf-cf4d-46bc-8ede-ce35d54d306f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 120ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default    

In [9]:
%%time
(
    spark.read.format("parquet")
    .load(f"{Path.home()}/data/G1_1e9_1e2_0_0.parquet")
    .where("id1='id016'")
    .groupBy("id1")
    .sum("v1")
    .collect()
)

CPU times: user 104 ms, sys: 29.7 ms, total: 134 ms
Wall time: 57.7 s


[Row(id1='id016', sum(v1)=30003304)]

## pandas syntax

In [4]:
import pandas as pd

In [7]:
(
    pd.read_parquet(
        "~/data/G1_1e9_1e2_0_0.parquet",
        columns=["id1", "v1"],
        filters=[("id1", "=", "id016")],
    )
    .query("id1 == 'id016'")
    .groupby("id1")
    .agg({"v1": "sum"})
)

,v1
id1,
id016,30003304
